In [14]:
import urllib3
import requests
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import time
import json
import os
from persiantools import characters

In [15]:
pd.options.display.max_rows = 400

In [16]:
def rows_text(rows):

    nemad=rows[0].text.strip()

    company_name=rows[1].text.strip()

    title=rows[3].text.strip()

    title_code=rows[4].text.strip()

    send_time=rows[5].text.strip()

    publish_time=rows[6].text.strip()

    href_html=rows[7].find_elements_by_tag_name('a')[0].get_attribute('href').strip()

    dict_1={'nemad':[nemad],
            'company_name':[company_name],
            'title':[title],
            'title_code':[title_code],
            'send_time':[send_time],
            'publish_time':[publish_time],
            'href_html':[href_html]}
    return dict_1

In [17]:
def url_df(nemad_name,page_number,driver_exe):
    url_1=f'https://www.codal.ir/ReportList.aspx?search&Symbol={nemad_name}&LetterType=6&Isic=571924&AuditorRef=-1&PageNumber={page_number}&Audited&NotAudited&IsNotAudited=false&Childs=false&Mains&Publisher=false&CompanyState=0&Category=-1&CompanyType=1&Consolidatable&NotConsolidatable'

    ch_option=webdriver.ChromeOptions()
    ch_option.add_argument('--headless')
    ch_option.add_argument('--no-sandbox')
    ch_option.add_argument('--disable-dev-shm-usage')

    driver_1=webdriver.Chrome(driver_exe,options=ch_option)

    driver_1.get(url_1)
    time.sleep(7)
    base_df=pd.DataFrame(columns=['nemad', 'company_name', 'title', 'title_code', 'send_time', 'publish_time', 'href_html'])

    for i in driver_1.find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[2]/table/tbody').find_elements_by_tag_name('tr'):
        row_1=i.find_elements_by_tag_name('td')
        base_df=pd.concat([base_df,pd.DataFrame(rows_text(row_1))])
    return base_df

In [18]:
def page_count(url,driver_exe):
    ch_option=webdriver.ChromeOptions()
    ch_option.add_argument('--headless')
    ch_option.add_argument('--no-sandbox')
    ch_option.add_argument('--disable-dev-shm-usage')
    driver_2=webdriver.Chrome(driver_exe,options=ch_option)
    driver_2.get(url)
    time.sleep(7)
    try:
        page_number=len(driver_2.\
                        find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[2]/nav/paging/ul').\
                        find_elements_by_tag_name('li'))-5
    except:
        page_number=1
    return page_number

In [19]:
def nemad_info(nemad:str,driver_exe=r'E:\Scrape\Driver\chromedriver.exe'):
    url=f'https://www.codal.ir/ReportList.aspx?search&Symbol={nemad}&LetterType=6&Isic=571924&AuditorRef=-1&PageNumber=1&Audited&NotAudited&IsNotAudited=false&Childs=false&Mains&Publisher=false&CompanyState=0&Category=-1&CompanyType=1&Consolidatable&NotConsolidatable'
    
    final_df=pd.DataFrame(columns=['nemad', 'company_name', 'title', 'title_code', 'send_time', 'publish_time', 'href_html'])

    for j in list(range(1,page_count(url,driver_exe)+1)):
        final_df=pd.concat([final_df,url_df(nemad,j,driver_exe)])
    final_df.reset_index(drop=True,inplace=True)

    final_df = final_df[final_df['title'].str.contains('سال مالی')]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.head(12)
    
    return final_df

In [20]:
# melat=nemad_info('شبندر')
# melat

In [21]:
def tables(url,driver_exe):

    ch_option=webdriver.ChromeOptions()
    ch_option.add_argument('--headless')
    ch_option.add_argument('--no-sandbox')
    ch_option.add_argument('--disable-dev-shm-usage')

    driver=webdriver.Chrome(driver_exe,options=ch_option)
    print('1')
    def check_th_exist(row):
        try :
            row.find_element_by_tag_name('th')
            print('2')
            return True
        except:
            print('3')
            return False

    def check_value_tag(var):
        if (var.text=='')|(var.text==None):
            try:

                x=var.get_attribute('value')
                print('4')
                return x
            except:
                x=var.text
                print('5')
                return x.strip()
        else:
            x=var.text
            print('6')
            return x.strip()


    row_list=[]

    T=0
    while T<5:
        try :
            driver.get(url)
            print('7')
            for row in driver.find_elements_by_tag_name('tr'):
                if check_th_exist(row) == True:
                    print('8')
                    r=[]
                    for th in row.find_elements_by_tag_name('th'):
                        r.append(check_value_tag(th))

                    row_list.append(r)
                rr=[]
                for td in row.find_elements_by_tag_name('td'):

                    rr.append(check_value_tag(td))
                row_list.append(rr)

            T=6
        except:
            T+=1
    print('c-1')
    df=pd.DataFrame(row_list)
    print('c-2')
    return df

In [22]:

def insert_table_title(df_2,i,path_3,row_base):
    def add_col(dff):
        dff['nemad']=row_base.iloc[0]
        dff['company_name']=row_base.iloc[1]
        dff['title']=row_base.iloc[2]
        dff['send_time']=row_base.iloc[4]
        dff['publish_time']=row_base.iloc[5]
        return dff
    try:

        if i=='12':
            df_2=add_col(df_2)
            df_2['table_title']='Consolidated Balance Sheet'
            df_2.to_excel(path_3+'\\Consolidated Balance Sheet.xlsx')
        if i == '13':
            df_2=add_col(df_2)
            df_2['table_title']='Consolidated Income Statement'
            df_2.to_excel(path_3+'\\Consolidated Income Statement.xlsx')
        if i == '14':
            df_2=add_col(df_2)
            df_2['table_title']='Consolidated Cash Flow'
            df_2.to_excel(path_3+'\\Consolidated Cash Flow.xlsx')
        if i == '0':
            print('c-3')
            df_2=add_col(df_2)
            print('c-4')
            df_2['table_title']='Balance Sheet'
            print('c-5')
            df_2.to_excel(path_3+'\\Balance Sheet.xlsx')
            print('c-6')
        if i == '1':
            df_2=add_col(df_2)
            df_2['table_title']='Income Statement'
            df_2.to_excel(path_3+'\\Income Statement.xlsx')
        if i == '9':
            df_2=add_col(df_2)
            df_2['table_title']='Cash Flow'
            df_2.to_excel(path_3+'\\Cash Flow.xlsx')
    except:
        pass

In [23]:
def get_tables(df_url,dirr=r'E:\Scrape\Files',driver_exe=r'E:\Scrape\Driver\chromedriver.exe'):
    
    path=dirr+f"\\{df_url.loc[0,'nemad']}"
    is_exist_1=os.path.exists(path)
    if not is_exist_1:
        os.makedirs(path)

    w=1
    
    for index,row in df_url.iterrows():
        path_2=path+f"\\{row.loc['nemad']}-{w}"
        is_exist_2=os.path.exists(path_2)
        if not is_exist_2:
            os.makedirs(path_2)
        w+=1
        
        ch_option=webdriver.ChromeOptions()
        ch_option.add_argument('--headless')
        ch_option.add_argument('--no-sandbox')
        ch_option.add_argument('--disable-dev-shm-usage')
        
        driver=webdriver.Chrome(driver_exe,options=ch_option)
        TT=0
        while TT<5:
            try:
                driver.get(row['href_html'])
                time.sleep(5)
                
                def check_value(driver):
                    values=driver.find_element_by_xpath('/html/body/form/div[4]/div[3]/select').\
                    find_elements_by_tag_name('option')

                    value_list=[v.get_attribute('value') for v in values]
                    return value_list
                target_page_list=['0','1']
                
                for i in target_page_list:
           
                    if i in check_value(driver):
                        
                        url_2=f"{row['href_html']}"+f'&SheetId={i}'
                        
                        insert_table_title(tables(url_2,driver_exe),i,path_2,row)
                        
                    else:
                        
                        pass
                TT=6
            except:
                TT+=1

In [24]:
nemad_file=pd.read_excel(r'E:\Scrape\Nemad Link\nemad link.xlsx')
df_nemad_11 = pd.read_excel(r'E:\Scrape\Nemad Link\ScrapedCashFlowListAndUpdateBSAndPL_Abbasi_14011020.xlsx')

nemad_file['nemad'] = nemad_file['nemad'].apply(lambda x : characters.ar_to_fa(x))
df_nemad_11['stock_name_x'] = df_nemad_11['stock_name_x'].apply(lambda x : characters.ar_to_fa(x))

nemad_file_2 = nemad_file[nemad_file['nemad'].isin(df_nemad_11['stock_name_x'])]
nemad_file_2.reset_index(drop=True , inplace=True)


In [25]:
nemad_file_1=nemad_file_2[(nemad_file_2.index<113)&(nemad_file_2.index>=80)]
nemad_file_1

,nemad,page-link
80,ثفارس,http://www.tsetmc.com/loader.aspx?ParTree=1513...
81,تاپیکو,http://www.tsetmc.com/loader.aspx?ParTree=1513...
82,بایکا,http://www.tsetmc.com/loader.aspx?ParTree=1513...
83,کفپارس,http://www.tsetmc.com/loader.aspx?ParTree=1513...
84,پتایر,http://www.tsetmc.com/loader.aspx?ParTree=1513...
85,کگل,http://www.tsetmc.com/loader.aspx?ParTree=1513...
86,فروی,http://www.tsetmc.com/loader.aspx?ParTree=1513...
87,حآفرین,http://www.tsetmc.com/loader.aspx?ParTree=1513...
88,دجابر,http://www.tsetmc.com/loader.aspx?ParTree=1513...
89,اپال,http://www.tsetmc.com/loader.aspx?ParTree=1513...


In [30]:
def internet_on():
    try:
        requests.get('http://google.com',timeout=2)
        return True
    except:
        return False
    

In [31]:

for index_1,row_1 in nemad_file_1.iterrows():
    B=2
    while B==2:
        if internet_on():
            df_1=nemad_info(row_1.loc['nemad'])
            if df_1.empty:
                continue
            else:
                get_tables(df_1,dirr=r'E:\Scrape\Files\tsetmc-data')
            B=1
        else:
            B=2

C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\1868218905.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_2=webdriver.Chrome(driver_exe,options=ch_option)
C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\1868218905.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  find_element_by_xpath('/html/body/form/div[3]/div[1]/div[1]/div[2]/div[2]/nav/paging/ul').\
c:\Users\abyo0503\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\285328825.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_1=webdriver.Chrome(driver_exe,options=ch_op

1
7
2
8
6
6
6
6
4
6
2
8
6
6
6
3
6


C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\2933190445.py:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for row in driver.find_elements_by_tag_name('tr'):
c:\Users\abyo0503\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


4
4
4
4
4
3
6
4
4
4
4
4
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
4
4
4
4
4
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
4
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
4
4
4
4
4
3
6
4
4
4
4
4
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
4
4
4
4
4
3
6
4
4
4
4
4
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
4
4
4
4
4
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
4
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
3
6
6
6
6
4
6
c-1
c-2
c-3
c-4
c-5
c-6


C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\139454268.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  values=driver.find_element_by_xpath('/html/body/form/div[4]/div[3]/select').\
c:\Users\abyo0503\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\2933190445.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(driver_exe,options=ch_option)


...

C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\139454268.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  values=driver.find_element_by_xpath('/html/body/form/div[4]/div[3]/select').\
c:\Users\abyo0503\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py:359: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\2933190445.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(driver_exe,options=ch_option)


1
7
2
8
6
6
4
6
4
6
2
8
6
4
6
3


C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\2933190445.py:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for row in driver.find_elements_by_tag_name('tr'):
c:\Users\abyo0503\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
7
2
8
6
6
4
6
4
6
2
8
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
4
4
4
4
4
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
6
3
6
6
4
6
4
4
3
4
4
4
4
4
4
3
6
3
4
3
4
3
4
3
6
3
4
c-1
c-2


C:\Users\abyo0503\AppData\Local\Temp\ipykernel_15536\139454268.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(driver_exe,options=ch_option)
